In [ ]:
# 有问题的
#  20210715_1627_w1 20210715_1636_w2 20210715_1648_w4

#  20210715_1659_w1w: Items of 'xdata' vector must satisfy the condition: x1 < x2 < ... < xN

In [17]:
import os
import time
from Lib.YamlFrames import OpenYaml, GetWormName,Just_Get_Raw,Get_First_Frames,Get_End_Frames,Extract_Yaml_Multiprocess_one

time_start = time.time()

workpath = '/home/data2/WormPoseData/ProcessingData/Temp/20210715_1722_w5w'

wormname = workpath.split('/')[-1]
print("Extract:",wormname)
# 改变这行填入你的Yaml文件地址
yaml_path = os.path.join(workpath,wormname,wormname+'.yaml')



ListFile = OpenYaml(yaml_path) # 导入数据

ListFile[-1]
linenumber = ListFile[-99].split(':')[-1]


begin_num1 = Get_First_Frames(ListFile)
end_num = Get_End_Frames(ListFile)
All_Frames_num = end_num-begin_num1+1  # 总的帧数
print("Total frames:",All_Frames_num)

# YamlData = Just_Get_Raw(yaml_path)
YamlData = Extract_Yaml_Multiprocess_one(yaml_path)



time_end = time.time()
print('time cost:',time_end-time_start,'s')



Extract: 20210715_1722_w5w
Total frames: 12095
Total frames: 12095
Os is Linux
 Thu Jul 15 17:22:31 2021
Load data time cost: 3.5456066131591797 s
并行处理数据 time cost: 5.045753002166748 s
time cost: 9.260370969772339 s
